Imports

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


Dataset preparation and tokenising

In [2]:
dataset = load_dataset("tatsu-lab/alpaca")

Using custom data configuration tatsu-lab--alpaca-2b32f0433506ef5f
Found cached dataset parquet (/home/arjun/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset['train'].features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [4]:
dataset = dataset['train']
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [5]:
new_dataset = {'input':[], 'output':[]}
tokeniser = AutoTokenizer.from_pretrained('gpt2')

In [6]:
for data in dataset:
    input = data['text'].replace('\n','')
    input_list = input.split('###')
    new_dataset['input'].append(tokeniser(input_list[0]+input_list[1],truncation=True))
    new_dataset['output'].append(tokeniser(input_list[2],truncation=True)) 
new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format('torch')

In [7]:
new_dataset.features

{'input': {'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)},
 'output': {'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}}

In [8]:
# Making dataset smaller for fast training
new_dataset = new_dataset.select(range(1000))

In [9]:
new_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 1000
})

DataLoader

In [10]:
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=8)

Loading model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("gpt2")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Optimiser and scheduler

In [12]:
num_epochs = 10

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [19]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)

cuda


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.